In [152]:
from summa import summarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import os
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
from sumy.nlp.tokenizers import Tokenizer
import re
from rouge import Rouge

rouge = Rouge()


In [153]:
# Define functions for TextRank, LSA, and Relevance sentence scoring
LANGUAGE = "english"
SENTENCES_COUNT = 3
# TextRank Summarization

In [154]:
def textrank_summarize(text):
    # Create a parser for the input text
    LANGUAGE = "english"  # Set the language of the text
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = TextRankSummarizer(stemmer)
    
    num_sentences_to_select = 5  # Adjust the number of sentences
    summarizer.stop_words = get_stop_words(LANGUAGE)
    summarized_text = summarizer(parser.document, SENTENCES_COUNT)
    summarized_text = '\n'.join([str(sentence) for sentence in summarized_text])
    return summarized_text

In [155]:

# LSA Summarization
def lsa_summarize(text):
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)
    summarizer = LsaSummarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)

    summarized_text = summarizer(parser.document, SENTENCES_COUNT)
    summarized_text = '\n'.join([str(sentence) for sentence in summarized_text])
    
    return summarized_text


In [156]:
# Relevance-Based Summarization
def relevance_summarize(text):
    sentences = text.split('.')

    # Create a TF-IDF vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
    
    # Calculate cosine similarity between sentences
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Choose the most relevant sentences (e.g., top 3 sentences)
    num_sentences_to_select = 3  # Adjust the number of sentences
    top_sentence_indices = similarity_matrix.argsort()[0][-num_sentences_to_select:]
    
    # Get the top sentences
    relevant_sentences = [sentences[i] for i in top_sentence_indices]
    
    # Create the summary by joining the selected sentences
    summarized_text = '. '.join(relevant_sentences)
    return summarized_text

In [157]:
dataset_folder = "data\\opinosis\\topics"
topic_file = "accuracy_garmin_nuvi_255W_gps.txt.data"
current_directory = os.getcwd()
topic_text = ""
with open(os.path.join(current_directory, dataset_folder, topic_file), 'r') as file:
        topic_text = file.read()

print(topic_text)

, and is very, very accurate .
 but for the most part, we find that the Garmin software provides accurate directions, whereever we intend to go .
 This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .
 It provides immediate alternatives if the route from the online map program was inaccurate or blocked by an obstacle .
 I've used other GPS units, as well as GPS built into cars   and to this day NOTHING beats the accuracy of a Garmin GPS .
 It got me from point A to point B with 100% accuracy everytime .
 It has yet to disappoint, getting me everywhere with 100% accuracy .
0 out of 5 stars Honest, accurate review, , PLEASE READ !
 Aside from that, every destination I've thrown at has been 100% accurate .
In closing, this is a fantastic GPS with some very nice features and is very accurate in directions .
 Plus, I've always heard that there are  quirks  with any GPS be

In [158]:
textrank_summary = textrank_summarize(topic_text)
lsa_summary = lsa_summarize(topic_text)
relevance_summary = relevance_summarize(topic_text)

print('\n####################################################')
print('Text Rank')
print('####################################################')
print(textrank_summary)

print('\n####################################################')
print('LSA Rank')
print('####################################################')
print(lsa_summary)

print('\n####################################################')
print('Relevence')
print('####################################################')
print(relevance_summary)



####################################################
Text Rank
####################################################
, and is very, very accurate .
0 out of 5 stars Inexpensive, accurate, plenty of features, August 6, 2009 The only glitch I have found so far is that the speed limits are not 100% accurate, although the GPS, amazingly, is able to very accurately tell you how fast your vehicle is moving .
,  Very Accurate but with one small glitch I found ,  I'll explain in the CONS This is a great GPS, it is so easy to use and it is always accurate .

####################################################
LSA Rank
####################################################
This function is not accurate if you don't leave it in battery mode say, when you stop at the Cracker Barrell for lunch and to play one of those trangle games with the tees .
I used it the day I bought it,   and then this morning, and as soon as it comes on it is  ready to navigate  The only downfall of this product, and the on

In [159]:
# Calculate ROUGE scores

summary_folder = "data\\Opinosis\\summaries"
current_topic = "accuracy_garmin_nuvi_255W_gps"
summary_file = "accuracy_garmin_nuvi_255W_gps.1.gold"
# topic_file = "accuracy_garmin_nuvi_255W_gps.2.gold"
# topic_file = "accuracy_garmin_nuvi_255W_gps.3.gold"
# topic_file = "accuracy_garmin_nuvi_255W_gps.4.gold"

current_directory = os.getcwd()
gold_summary = ""
with open(os.path.join(current_directory, summary_folder, current_topic, summary_file), 'r') as file:
        gold_summary = file.read()
print(gold_summary)


This unit is generally quite accurate.  
Set-up and usage are considered to be very easy. 
The maps can be updated, and tend to be reliable.



In [160]:

# str_textRank = '\n'.join(textrank_summary)
# str_lsaRank = '\n'.join(lsa_summary)
# str_Relenevence = '\n'.join(relevance_summary)

rouge_scores_text_rank = rouge.get_scores(textrank_summary, gold_summary)
rouge_scores_lsa = rouge.get_scores(lsa_summary, gold_summary)
rouge_scores_relevance = rouge.get_scores(relevance_summary, gold_summary)

# Print or save the ROUGE scores for each method
print(f"ROUGE-1 scores for TextRank: {rouge_scores_text_rank[0]['rouge-1']}")
print(f"ROUGE-2 scores for TextRank: {rouge_scores_text_rank[0]['rouge-2']}")
print(f"ROUGE-1 scores for LSA: {rouge_scores_lsa[0]['rouge-1']}")
print(f"ROUGE-2 scores for LSA: {rouge_scores_lsa[0]['rouge-2']}")
print(f"ROUGE-1 scores for Relevance Scoring: {rouge_scores_relevance[0]['rouge-1']}")
print(f"ROUGE-2 scores for Relevance Scoring: {rouge_scores_relevance[0]['rouge-2']}")

ROUGE-1 scores for TextRank: {'r': 0.4090909090909091, 'p': 0.14285714285714285, 'f': 0.21176470204567477}
ROUGE-2 scores for TextRank: {'r': 0.0, 'p': 0.0, 'f': 0.0}
ROUGE-1 scores for LSA: {'r': 0.36363636363636365, 'p': 0.09411764705882353, 'f': 0.14953270701371307}
ROUGE-2 scores for LSA: {'r': 0.0, 'p': 0.0, 'f': 0.0}
ROUGE-1 scores for Relevance Scoring: {'r': 0.18181818181818182, 'p': 0.19047619047619047, 'f': 0.18604650663061129}
ROUGE-2 scores for Relevance Scoring: {'r': 0.0, 'p': 0.0, 'f': 0.0}
